In [47]:
import numpy as np
import pandas as pd

In [48]:
from typing import List, Dict

In [49]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [50]:
songs = pd.read_csv('E:/music_recommender-master/content based recommedation system/songdata.csv')

In [51]:
songs.head()

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \nAnd..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \nTouch me gentl..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \nWhy I had t...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [52]:
songs = songs.sample(n=5000).drop('link', axis=1).reset_index(drop=True)

In [53]:
songs['text'] = songs['text'].str.replace(r'\n', '')

In [54]:
tfidf = TfidfVectorizer(analyzer='word', stop_words='english')

In [55]:
lyrics_matrix = tfidf.fit_transform(songs['text'])

In [56]:
cosine_similarities = cosine_similarity(lyrics_matrix) 

In [57]:
similarities = {}

In [58]:
for i in range(len(cosine_similarities)):
    # Now we'll sort each element in cosine_similarities and get the indexes of the songs. 
    similar_indices = cosine_similarities[i].argsort()[:-50:-1] 
    # After that, we'll store in similarities each name of the 50 most similar songs.
    # Except the first one that is the same song.
    similarities[songs['song'].iloc[i]] = [(cosine_similarities[i][x], songs['song'][x], songs['artist'][x]) for x in similar_indices][1:]

In [59]:
class ContentBasedRecommender:
    def __init__(self, matrix):
        self.matrix_similar = matrix

    def _print_message(self, song, recom_song):
        rec_items = len(recom_song)
        
        print(f'The {rec_items} recommended songs for {song} are:')
        for i in range(rec_items):
            print(f"Number {i+1}:")
            print(f"{recom_song[i][1]} by {recom_song[i][2]} with {round(recom_song[i][0], 3)} similarity score") 
            print("--------------------")
        
    def recommend(self, recommendation):
        # Get song to find recommendations for
        song = recommendation['song']
        # Get number of songs to recommend
        number_songs = recommendation['number_songs']
        # Get the number of songs most similars from matrix similarities
        recom_song = self.matrix_similar[song][:number_songs]
        # print each item
        self._print_message(song=song, recom_song=recom_song)

In [60]:
recommedations = ContentBasedRecommender(similarities)

In [61]:
recommendation = {
    "song": songs['song'].iloc[10],
    "number_songs": 4 
}

In [67]:
def diversity_score(recommendations):
    unique_artists = set()
    for rec in recommendations.values():
        for _, song, artist in rec:
            unique_artists.add(artist)
    return len(unique_artists) / len(recommendations)

#Calculate diversity score for the recommendations
diversity = diversity_score(similarities)
print(f"Diversity Score: {diversity}")


Diversity Score: 0.1276282590412111


In [62]:
recommedations.recommend(recommendation)

The 4 recommended songs for Born To Be Bad are:
Number 1:
Bad Bad Boy by Nazareth with 0.423 similarity score
--------------------
Number 2:
I Got It Bad by Ella Fitzgerald with 0.335 similarity score
--------------------
Number 3:
I'm A Bad Bad Man by Irving Berlin with 0.318 similarity score
--------------------
Number 4:
Born A Man by Bee Gees with 0.313 similarity score
--------------------


And we can pick another random song and recommend again:

In [63]:
recommendation2 = {
    "song": songs['song'].iloc[120],
    "number_songs": 4 
}

In [65]:
recommedations.recommend(recommendation2)

The 4 recommended songs for Green Day are:
Number 1:
Do You Like It, Do You Want It by Puff Daddy with 0.267 similarity score
--------------------
Number 2:
I Don't Think We Could Ever Be Friends by Dusty Springfield with 0.14 similarity score
--------------------
Number 3:
I Think I Love You by Perry Como with 0.138 similarity score
--------------------
Number 4:
I Can't Give You Anything But Love by Ella Fitzgerald with 0.12 similarity score
--------------------


In [69]:
def precision_at_k(actual_songs, recommended_songs, k):
    actual_set = set(actual_songs)
    recommended_set = set(recommended_songs[:k])
    correct_set = actual_set.intersection(recommended_set)
    return len(correct_set) / k

actual_songs = ['song1', 'song2', 'song3']  # List of songs the user actually liked
recommended_songs = ['song4', 'song2', 'song5', 'song1']  # List of songs recommended to the user
k = 3  # Number of recommended songs to consider

precision = precision_at_k(actual_songs, recommended_songs, k)
print(f"Precision@{k}: {precision}")


Precision@3: 0.3333333333333333
